## Data modeling with MLlib

In this notebook, MLlib functions and classes are used to develop a machine learning application for predicting a binary response variable. MLlib works on top of **Spark** technology, so all data manipulation, pre-processing and modeling here are implemented using **distributed computing**. Some codes are extracted from the previous notebook, "Data Manipulation with PySpark". Again, the code paradigm is procedural for demonstrating how to use Spark APIs for machine learning. Both of these notebooks aim to illustrate how to deal with **big data** through Python language. Even that Spark (raw) API slightly differs to PySpark, codes here can be easily extended to other approaches for dealing with large amounts of data.

MLlib has a comprehensive API for all steps of model development: from [data pre-processing](https://spark.apache.org/docs/latest/ml-features.html) ([extraction](https://spark.apache.org/docs/latest/ml-features.html#feature-extractors), [transformation](https://spark.apache.org/docs/latest/ml-features.html#feature-transformers) and [selection](https://spark.apache.org/docs/latest/ml-features.html#feature-selectors) of features) to [model training](https://spark.apache.org/docs/latest/ml-classification-regression.html) and [tuning](https://spark.apache.org/docs/latest/ml-tuning.html), besides of model evaluation. The construction of [pipelines](https://spark.apache.org/docs/latest/ml-pipeline.html) is very straightforward with MLlib, as estimators and transformers are sequentially declared.

This notebook covers most of these topics: first, data is read and split into training and test sets; then, several data pre-processing steps are implemented, using direct calculations through SQL or making use of classes from MLlib. Finally, model estimation takes place: default and optimized models are trained based on logistic regression and GBM algorithms.

--------------

## Libraries

In [0]:
import pandas as pd
import numpy as np

from pyspark.sql import functions as func
from pyspark.sql.types import TimestampType

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

## Settings

In [0]:
# Declare whether outcomes should be exported:
export = False

# Number of unique values above which log transformation is applied:
thres_to_log = 100

# Share of missings above which features are disregarded:
thres_missings = 0.95

# Share of data above which categories are kept to one-hot encoding:
thres_cat_repr = 0.01

## Importing data

In [0]:
df = spark.read.format('csv').\
           options(header='true', delimiter = ',', inferSchema='true').\
           load("/FileStore/shared_uploads/matheusf.rosso@gmail.com/fraud_data_sample.csv")

# Sorting the dataframe by date:
df = df.sort(func.col('epoch').asc())

# Converting epoch into datetime:
df = df.withColumn('epoch', (func.col('epoch')/func.lit(1000)))
df = df.withColumn('datetime', func.date_format(df.epoch.cast(dataType=TimestampType()), "yyyy-MM-dd HH:mm:ss"))

first_date = df.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['first_date']
last_date = df.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['last_date']

print(f'Type of df: {type(df)}.')
print(f'Shape of df: ({df.count()}, {len(df.columns)}).')
print(f'Number of unique instances: {df.select("order_id").distinct().count()}.')
print(f'Time interval: ({first_date}, {last_date}).')

# Support variables:
drop_vars = ['y', 'order_amount', 'store_id', 'order_id', 'status', 'epoch', 'datetime', 'weight']

# df.display()

Type of df: <class 'pyspark.sql.dataframe.DataFrame'>.
Shape of df: (8361, 1429).
Number of unique instances: 8361.
Time interval: (2021-05-17 15:01:00, 2021-06-27 23:55:01).


### Train-test split

In [0]:
# Number of instances by date:
orders_by_date = df.withColumn('date', func.to_date(func.col('datetime'))).\
                    select('date', 'order_id').groupBy('date').agg(func.count('order_id').alias('freq'))
orders_by_date = orders_by_date.toPandas().sort_values('date', ascending=True)

# Accumulated number of instances by date:
orders_by_date['acum'] = np.cumsum(orders_by_date.freq)
orders_by_date['acum_share'] = [a/orders_by_date['acum'].max() for a in orders_by_date['acum']]

# Date with 75% of volume of data:
last_date_train = orders_by_date.iloc[np.argmin(abs(orders_by_date['acum_share'] - 0.75))]['date']

# Train-test split:
df_test = df.filter(func.col('datetime') > last_date_train)
df_train = df.filter(func.col('datetime') <= last_date_train)

# Instances identification for training and test data:
train_orders, test_orders = ([r['order_id'] for r in df_train.select('order_id').collect()],
                             [r['order_id'] for r in df_test.select('order_id').collect()])

first_date_train = df_train.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['first_date']
last_date_train = df_train.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['last_date']
first_date_test = df_test.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['first_date']
last_date_test = df_test.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['last_date']

print(f'Shape of df_train: ({df_train.count()}, {len(df_train.columns)}).')
print(f'Number of unique instances (training data): {df_train.select("order_id").distinct().count()}.')
print(f'Time interval (training data): ({first_date_train}, {last_date_train}).\n')

print(f'Shape of df_test: ({df_test.count()}, {len(df_test.columns)}).')
print(f'Number of unique instances (test data): {df_test.select("order_id").distinct().count()}.')
print(f'Time interval (test data): ({first_date_test}, {last_date_test}).')

# df_train.display()

Shape of df_train: (6313, 1429).
Number of unique instances (training data): 6313.
Time interval (training data): (2021-05-17 15:01:00, 2021-06-09 23:56:06).

Shape of df_test: (2048, 1429).
Number of unique instances (test data): 2048.
Time interval (test data): (2021-06-10 00:01:08, 2021-06-27 23:55:01).


## Data pre-processing

In [0]:
# Creating a temporary view table (training data):
df_train.createOrReplaceTempView("training_data")

# Creating a temporary view table (test data):
df_test.createOrReplaceTempView("test_data")

### Data types of features

In [0]:
# Lists with categorical and numerical variables:
cat_vars = [t[0] for t in df_train.dtypes if (t[1]=='string') & (t[0] not in drop_vars)]
num_vars = [t[0] for t in df_train.dtypes if (t[1]!='string') & (t[0] not in drop_vars)]

# Number of unique values for a sample of data:
unique_values_sample = df_train.select(num_vars).sample(fraction=0.05).toPandas().nunique()

# Name of numerical variables with sufficient variation for log transformation:
unique_values_sample = pd.DataFrame(data={
  'feature': unique_values_sample.index, 'num_uniques': unique_values_sample.values
})
to_log = list(unique_values_sample[unique_values_sample.num_uniques>thres_to_log]['feature'])

### Assessing missing values

#### Training data

In [0]:
# Dataframe with frequency of missings by feature:
missings_df_train = df_train.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_train.columns])

# Converting the dataframe into pandas for ready use:
missings_df_train = missings_df_train.toPandas().T.reset_index(drop=False)
missings_df_train.columns = ['feature', 'missings']

# Share of observations with missing value for each feature:
num_obs_train = df_train.count()
missings_df_train['share'] = missings_df_train['missings'].apply(lambda x: x/num_obs_train)

# List of variables with missings:
vars_missings_train = list(missings_df_train[missings_df_train.missings > 0]['feature'])

print(f'Number of features with missings: {sum(missings_df_train["missings"] > 0)} ({np.nanmean(missings_df_train["missings"] > 0)*100:.2f}%).')
print(f'Average number of missings: {missings_df_train["missings"].mean():.0f} ({(missings_df_train["missings"].mean()/num_obs_train)*100:.2f}%).')
# missings_df_train.sample(10)

Number of features with missings: 291 (20.36%).
Average number of missings: 353 (5.59%).


#### Test data

In [0]:
# Dataframe with frequency of missings by feature:
missings_df_test = df_test.select([func.count(func.when(func.col(c).isNull(), c)).alias(c) for c in df_test.columns])

# Converting the dataframe into pandas for ready use:
missings_df_test = missings_df_test.toPandas().T.reset_index(drop=False)
missings_df_test.columns = ['feature', 'missings']

# Share of observations with missing value for each feature:
num_obs_test = df_test.count()
missings_df_test['share'] = missings_df_test['missings'].apply(lambda x: x/num_obs_test)

print(f'Number of features with missings: {sum(missings_df_test["missings"] > 0)} ({np.nanmean(missings_df_test["missings"] > 0)*100:.2f}%).')
print(f'Average number of missings: {missings_df_test["missings"].mean():.0f} ({(missings_df_test["missings"].mean()/num_obs_test)*100:.2f}%).')
# missings_df_test.sample(10)

Number of features with missings: 164 (11.48%).
Average number of missings: 102 (4.97%).


### Early selection of features

In [0]:
# Filtering variables with excessive number of missings:
excessive_missings_train = list(missings_df_train[missings_df_train.share > thres_missings]['feature'])

# Variance of each numerical variable:
variance_train = df_train.select([func.variance(func.col(c)).alias(c) for c in num_vars])

# Converting the dataframe into pandas for ready use:
variance_train = variance_train.toPandas().T.reset_index(drop=False)
variance_train.columns = ['feature', 'variance']

# Filtering variables with no variance:
no_variance_train = list(variance_train[variance_train.variance.apply(lambda x: round(x, 6))==0]['feature'])

In [0]:
initial_vars = df_train.select([c for c in df_train.columns if c not in drop_vars]).columns
print(f'Initial number of features: {len(initial_vars)}.')
print(f'{len(excessive_missings_train)} features were dropped for excessive number of missings!')
print(f'{len(no_variance_train)} features were dropped for having no variance!')
print(f'{len(initial_vars) - len(excessive_missings_train) - len(no_variance_train)} remaining features')

disregard_vars = []
disregard_vars.extend(excessive_missings_train)
disregard_vars.extend(no_variance_train)

# Dropping variables with exceissive number of missings or no variance:
num_vars = [c for c in num_vars if c not in disregard_vars]
to_log = [c for c in to_log if c not in disregard_vars]
cat_vars = [c for c in cat_vars if c not in disregard_vars]

Initial number of features: 1421.
3 features were dropped for excessive number of missings!
361 features were dropped for having no variance!
1057 remaining features


### Logarithmic transformation

In [0]:
# Query clause for log transforming numerical data:
log_transf = "CASE WHEN `{feat}` IS NOT NULL THEN ln(`{feat}`+0.0001) ELSE 0 END AS `L#{feat}`"
log_transf = ', '.join([log_transf.format(feat=c) for c in to_log])

### Standard scale transformation (first alternative)

This first approach to generate data standard scaled considers the following order of operations: logarithmic transformation, standard scaling, missing values treatment and transformation of categorical variables. The first three procedures are implemented through SQL, and only the last one uses MLlib API.

In [0]:
# # Clause for extracting numerical variables:
# num_vars_clause = "`{feat}`"
# num_vars_clause = ', '.join([num_vars_clause.format(feat=c) for c in num_vars if c not in to_log])

# # Clause for extracting numerical variables that should be log-transformed:
# log_vars_clause = "CASE WHEN `{feat}` IS NOT NULL THEN ln(`{feat}`+0.0001) ELSE `{feat}` END AS `L#{feat}`"
# log_vars_clause = ', '.join([log_vars_clause.format(feat=c) for c in to_log])

# # Querying numerical variables from the training data:
# training_num_data = spark.sql(f'SELECT order_id, {log_vars_clause}, {num_vars_clause} FROM training_data')

In [0]:
# # Mean and standard deviation of each numerical variable:
# mean_train = training_num_data.select([func.mean(func.col(c)).alias(c) for c in [c for c in training_num_data.columns if c!='order_id']])
# std_train = training_num_data.select([func.stddev(func.col(c)).alias(c) for c in [c for c in training_num_data.columns if c!='order_id']])

# # Converting dataframes into dictionaries for ready use:
# mean_train = mean_train.toPandas().T.reset_index(drop=False)
# mean_train.columns = ['feature', '_mean']
# mean_train['_mean'] = mean_train['_mean'].apply(lambda x: x*-1)
# mean_train['sign'] = mean_train['_mean'].apply(lambda x: '-' if x < 0 else '+')
# sign_train = dict(zip(mean_train['feature'], mean_train['sign']))
# mean_train = dict(zip(mean_train['feature'], [abs(x) for x in mean_train['_mean']]))

# std_train = std_train.toPandas().T.reset_index(drop=False)
# std_train.columns = ['feature', 'std']
# std_train = dict(zip(std_train['feature'], std_train['std']))

# # Query clause for log transforming and standardizing numerical data:
# log_transf = "CASE WHEN `{feat}` IS NOT NULL THEN (ln(`{feat}`+0.0001){sign}{avg})/{std} ELSE 0 END AS `L#{feat}`"
# log_transf = ', '.join([log_transf.format(feat=c, avg=round(mean_train[f'L#{c}'], 4), sign=sign_train[f'L#{c}'], std=round(std_train[f'L#{c}'], 4)) for
#                         c in to_log])

# # Query clause to standardize numerical variables and impute missings (except from those that had already been treated during log transformation):
# impute_missings_num = "CASE WHEN `{feat}` IS NULL THEN 0 ELSE (`{feat}`{sign}{avg})/{std} END AS `{feat}`"
# impute_missings_num = ', '.join([impute_missings_num.format(feat=c, avg=round(mean_train[c], 4), sign=sign_train[c], std=round(std_train[c], 4)) for
#                                  c in num_vars if c not in to_log])

### Treating missing values

#### Categorical variables

In [0]:
repr_categories = {}

# Loop over categorical variables:
for c in cat_vars:
  # Categories whose share is larger than 0.01:
  repr_categories[c] = [r[c] for r in df_train.groupBy(c).\
                                               agg(func.countDistinct(func.col('order_id'))).\
                                               withColumn('share', (func.col('count(order_id)')/func.lit(num_obs_train))).\
                                               filter(func.col('share')>thres_cat_repr).select(c).collect()]
  
  # Handling missings and unique categories:
  repr_categories[c] = [c for c in repr_categories[c] if pd.isna(c)==False]
  
  if len(repr_categories[c]) <= 1:
    repr_categories.pop(c)
    cat_vars = [v for v in cat_vars if v!=c]
  
# Query clause to impute missing values of categorical variables and treat their residual categories:
impute_missings_cat = "CASE WHEN `{feat}` IN {repr_cat} THEN `{feat}` WHEN `{feat}` IS NULL THEN 'missing_value' ELSE 'residual' END AS `{feat}`"
impute_missings_cat = ', '.join([impute_missings_cat.format(feat=c, repr_cat=tuple(repr_categories[c])) for c in cat_vars])

#### Numerical variables

In [0]:
# Query clause for creating binary variable indicating missing values:
missing_vars = "CASE WHEN `{feat}` IS NULL THEN 1 ELSE 0 END AS `NA#{feat}`"
missing_vars = ', '.join([missing_vars.format(feat=c) for c in num_vars if c in vars_missings_train])

# Query clause to impute missing values of numerical variables (except from those that had already been treated during log transformation):
impute_missings_num = "CASE WHEN `{feat}` IS NULL THEN 0 ELSE `{feat}` END AS `{feat}`"
impute_missings_num = ', '.join([impute_missings_num.format(feat=c) for c in num_vars if c not in to_log])

# Query clause with support variables:
drop_vars_query = ', '.join(drop_vars)

#### Training data

In [0]:
# Concatenating the query and running it to produce the transformed dataframe:
query = f'SELECT {drop_vars_query}, {log_transf}, {missing_vars}, {impute_missings_num}, {impute_missings_cat} FROM training_data'
df_train = spark.sql(query)

first_date_train = df_train.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['first_date']
last_date_train = df_train.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['last_date']

print(f'Shape of df_train: ({df_train.count()}, {len(df_train.columns)}).')
print(f'Number of unique instances (training data): {df_train.select("order_id").distinct().count()}.')
print(f'Time interval (training data): ({first_date_train}, {last_date_train}).')

# df_train.display()

Shape of df_train: (6313, 1330).
Number of unique instances (training data): 6313.
Time interval (training data): (2021-05-17 15:01:00, 2021-06-09 23:56:06).


#### Test data

In [0]:
# Concatenating the query and running it to produce the transformed dataframe:
query = f'SELECT {drop_vars_query}, {log_transf}, {missing_vars}, {impute_missings_num}, {impute_missings_cat} FROM test_data'
df_test = spark.sql(query)

first_date_test = df_test.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['first_date']
last_date_test = df_test.agg(func.min('datetime').alias('first_date'), func.max('datetime').alias('last_date')).collect()[0]['last_date']

print(f'Shape of df_test: ({df_test.count()}, {len(df_test.columns)}).')
print(f'Number of unique instances (test data): {df_test.select("order_id").distinct().count()}.')
print(f'Time interval (test data): ({first_date_test}, {last_date_test}).')

# df_test.display()

Shape of df_test: (2048, 1330).
Number of unique instances (test data): 2048.
Time interval (test data): (2021-06-10 00:01:08, 2021-06-27 23:55:01).


### Categorical features transformation

The transformation of categorical variables relies on two classes of MLlib API: [StringIndexer](https://spark.apache.org/docs/latest/ml-features.html#stringindexer) creates an estimator that, once fitted on training data, converts strings to label, while [OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder) works similarly, but converting categories into binary variables.

In [0]:
# Creating the object that will convert categories into labels:
string_indexer = StringIndexer(inputCols=cat_vars, outputCols=[x + "_Index" for x in cat_vars])

# Creating the object that will convert labels into binary variables:
encoder = OneHotEncoder(inputCols=string_indexer.getOutputCols(), outputCols=[x + "_OHE" for x in cat_vars])

In [0]:
# Visualizing the dataframe where categories are converted into labels:
stringIndexerModel = string_indexer.fit(df_train)
# display(stringIndexerModel.transform(df_train).take(3))

### Datasets consistency

In [0]:
if len(df_train.columns)!=len(df_test.columns) | len(df_train.columns)!=sum([train==test for train, test in zip(df_train.columns, df_test.columns)]):
  print('Training and test dataframes are inconsistent with each other!')

### Vector assembler

MLlib algorithms may require the creation of a single vector with all values of features for each data point. This can be done through the [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler) class.

In [0]:
features = [c for c in df_train.columns if (c not in drop_vars) & (c not in cat_vars)]
features.extend([c+'_OHE' for c in cat_vars])

# Creating the object that will create a features vector out of all features columns:
vec_assembler = VectorAssembler(inputCols=features, outputCol="features")

### Standard scaling (second alternative)

The second approach for standard scaling numerical data uses the [StandardScaler](https://spark.apache.org/docs/latest/ml-features.html#standardscaler) class of MLlib API. Here, all variables are standardized after logarithmic transformation, missing values treatment and one-hot encoding of categorical variables.

In [0]:
# Creating the object that standard scales the inputs:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

## Data modeling

### Default models

#### Logistic regression

Model estimation with MLlib, as done here training a [logistic regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression) model, depends on the creation of a [pipeline](https://spark.apache.org/docs/latest/ml-pipeline.html) that sequentially implements all necessary data processing operations.

In [0]:
# Creating the object of logistic regression model:
lr_model = LogisticRegression(featuresCol="scaledFeatures", labelCol="y", regParam=1.0)

# Creating the pipeline that sequentially fits and transforms the data:
pipeline_lr = Pipeline(stages=[string_indexer, encoder, vec_assembler, scaler, lr_model])
print(f'Type of pipeline_lr: {type(pipeline_lr)}.')

# Training the model by running through all steps of the pipeline:
pipeline_model_lr = pipeline_lr.fit(df_train)
print(f'Type of pipeline_model_lr: {type(pipeline_model_lr)}.')

Type of pipeline_lr: <class 'pyspark.ml.pipeline.Pipeline'>.
Type of pipeline_model_lr: <class 'pyspark.ml.pipeline.PipelineModel'>.


Predictions on test data

In [0]:
# Applying the pipeline over test data and generating predictions:
test_pred_lr = pipeline_model_lr.transform(df_test)
print(f'Type of test_pred_lr: {type(test_pred_lr)}.')
print(f'Shape of test_pred_lr: ({test_pred_lr.count(), len(test_pred_lr.columns)}).')

# display(test_pred_lr.select('features', 'y', 'prediction', 'probability').take(3))

Type of test_pred_lr: <class 'pyspark.sql.dataframe.DataFrame'>.
Shape of test_pred_lr: ((2048, 1349)).


Model evaluation

The [BinaryClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.BinaryClassificationEvaluator.html) class allows the calculation of either ROC-AUC or the area under the precision-recall curve.

In [0]:
# Creating the object that evaluates model performance:
roc_auc_eval = BinaryClassificationEvaluator(labelCol='y', metricName="areaUnderROC")

print(f"Test ROC-AUC: {roc_auc_eval.evaluate(test_pred_lr):.4f}")

Test ROC-AUC: 0.9040


#### GBM

Another learning algorithm available with MLlib is [GBM](https://spark.apache.org/docs/latest/ml-classification-regression.html#gradient-boosted-tree-classifier).

In [0]:
# Creating the object of GBM:
gbm_model = GBTClassifier(featuresCol="features", labelCol="y", maxDepth=3, stepSize=0.01, subsamplingRate=0.75)

# Creating the pipeline that sequentially fits and transforms the data:
pipeline_gbm = Pipeline(stages=[string_indexer, encoder, vec_assembler, gbm_model])
print(f'Type of pipeline_gbm: {type(pipeline_gbm)}.')

# Training the model by running through all steps of the pipeline:
pipeline_model_gbm = pipeline_gbm.fit(df_train)
print(f'Type of pipeline_model_gbm: {type(pipeline_model_gbm)}.')

Type of pipeline_gbm: <class 'pyspark.ml.pipeline.Pipeline'>.
Type of pipeline_model_gbm: <class 'pyspark.ml.pipeline.PipelineModel'>.


Predictions on test data

In [0]:
# Applying the pipeline over test data and generating predictions:
test_pred_gbm = pipeline_model_gbm.transform(df_test)
print(f'Type of test_pred_gbm: {type(test_pred_gbm)}.')
print(f'Shape of test_pred_gbm: ({test_pred_gbm.count(), len(test_pred_gbm.columns)}).')

# display(test_pred_gbm.select('features', 'y', 'prediction', 'probability').take(3))

Type of test_pred_gbm: <class 'pyspark.sql.dataframe.DataFrame'>.
Shape of test_pred_gbm: ((2048, 1348)).


Model evaluation

In [0]:
print(f"Test ROC-AUC: {roc_auc_eval.evaluate(test_pred_gbm):.4f}")

Test ROC-AUC: 0.8674


### Optimizing hyper-parameters

The optimization of hyper-parameters is available under MLlib through two classes: [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.ParamGridBuilder.html), which creates a grid with different combinations of values of hyper-parameters, and [CrossValidator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.tuning.CrossValidator.html?highlight=crossvalidator), which implements K-folds CV estimation.

#### Logistic regression

In [0]:
# Creating the object that defines a grid of hyper-parameters values:
paramGrid_lr = (ParamGridBuilder()
               .addGrid(lr_model.regParam, [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0])
               .build())

# Creating the object for running K-folds CV:
cv = CrossValidator(estimator=pipeline_lr, estimatorParamMaps=paramGrid_lr, evaluator=roc_auc_eval, numFolds=3, parallelism=4)

# Running K-folds CV estimation:
cv_lr = cv.fit(df_train)

/databricks/spark/python/pyspark/ml/util.py:838: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

Evaluating the final model on test data

In [0]:
# Predictions on test data from the best model:
test_pred_lr = cv_lr.transform(df_test)

# Evaluating model performance:
print(f"Test ROC-AUC: {roc_auc_eval.evaluate(test_pred_lr):.4f}")

#### GBM

In [0]:
# Creating the object that defines a grid of hyper-parameters values:
paramGrid_gbm = (ParamGridBuilder()
                .addGrid(gbm_model.maxDepth, [1, 3, 5])
                .addGrid(gbm_model.stepSize, [0.001, 0.01, 0.1])
                .addGrid(gbm_model.subsamplingRate, [0.5, 0.75, 1.0])
                .build())

# Creating the object for running K-folds CV:
cv_gbm = CrossValidator(estimator=pipeline_gbm, estimatorParamMaps=paramGrid_gbm, evaluator=roc_auc_eval, numFolds=3, parallelism = 4)
 
# Running K-folds CV estimation:
cv_gbm = cv_gbm.fit(df_train)

Evaluating the final model on test data

In [0]:
# Predictions on test data from the best model:
test_pred_gbm = cv_gbm.transform(df_test)

# Evaluating model performance:
print(f"Test ROC-AUC: {roc_auc_eval.evaluate(test_pred_gbm):.4f}")